## LSH For Disney Names Matching

### Functions

In [1]:
import logging
import pandas as pd
import argparse
import timeit
from datasketch import MinHash, MinHashLSH
from nltk import ngrams
from datetime import date, timedelta
import numpy as np

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__ + '.vaq')

In [3]:
def load_data():
    """
    Reads data from Excel file
    """
    df = pd.read_excel("disney_characters.xlsx")
    
    return df

In [4]:
def clean_data(df):
    """
    Cleans up string columns. Names should only contain letters.
    :param df: dataframe with Disney character names
    """

    logger = logging.getLogger(__name__ + '.clean_data')

    starttime = timeit.default_timer()

    df['local_name_clean'] = df.local_name.str.replace('[^a-zA-Z]', '', regex=True).str.lower()
   
    logger.info(f"Cleaning done after {round(timeit.default_timer() - starttime, 2)} s")

In [5]:
def minhash_data(df_column, threshold, num_perm, weights, nr_ngrams):
    """
    Performs hashing of columns using MinHash
    :param df_column: column to hash
    :param threshold: Jaccard distance threshold
    :param num_perm: number of permutations
    :param weights: false positive and false negative weights
    :param nr_ngrams: shingle size
    :return: lsh, minhash objects
    """

    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm, weights=weights)

    # Create MinHash objects
    minhashes = {}

    for c, i in enumerate(df_column):
        minhash = MinHash(num_perm=num_perm)

        for d in ngrams(i, nr_ngrams):
            minhash.update("".join(d).encode('utf-8'))
        lsh.insert(c, minhash)
        minhashes[c] = minhash

    return lsh, minhashes

In [6]:
def query_lsh(df, minhashes, lsh):
    """
    Queries records in df and if it finds a match appends to a new df. Giving the MinHash of the query set, 
    retrieve the keys that references sets with Jaccard similarities greater than the threshold.
    :param df: dataframe with Disney characters
    :param minhashes: minhashes of the name column
    :param lsh: lsh of the name column
    :return: df_matches: df with pairs of matched records
    """

    df_matches = pd.DataFrame(columns=['id', 'local_name', 'english_name', 'match_id', 'match_local_name', 
                                       'match_english_name'])

    for i in range(len(df)):
        result = lsh.query(minhashes[i])
        result = [element for element in result if element is not i]

        if len(result) > 0:
            df_i = pd.DataFrame()
            df_i['id'] = df.id[result]
            df_i['local_name'] = df.local_name[result]
            df_i['english_name'] = df.english_name[result]
            df_i['match_id'] = df.id[i]
            df_i['match_local_name'] = df.local_name[i]
            df_i['match_english_name'] = df.english_name[i]

            df_matches = df_matches.append(df_i)

    return df_matches

In [76]:
def lsh_match_name(df, threshold, num_perm, weights, nr_ngrams):
    """
    Creates minhashes on Disney character names. Using LSH returns pairs of matched names. 
    :param df: dataframe with Disney characters
    :param threshold: Jaccard distance threshold
    :param num_perm: number of permuntations
    :param weights: false positive and false negative weights
    :param nr_ngrams: shingle size
    :return:
    """

    logger = logging.getLogger(__name__ + '.lsh_match_name')
    logger.debug("Creating minhashes...")

    lsh, minhashes = minhash_data(df.local_name_clean, threshold=threshold, num_perm=num_perm, weights=weights, 
                                  nr_ngrams=nr_ngrams)

    logger.debug("Querying minhashes...")
    df_name = query_lsh(df, minhashes, lsh)
    
    df_name = df_name[df_name.id != df_name.match_id]

    return df_name

### Read data

In [8]:
df = load_data()
df.head()

,id,language,english_name,local_name
0,1,Danish,"April, May, and June","Kylle, Pylle og Rylle"
1,2,Danish,Beagle Boys,Bjørnebanden
2,3,Danish,Big Bad Wolf,Store stygge ulv
3,4,Danish,Black Pete,Sorteper
4,5,Danish,Chip 'n Dale,Chip og Chap


In [36]:
df.english_name.value_counts().head()

Donald Duck       20
Beagle Boys       20
Gyro Gearloose    20
Mickey Mouse      20
Minnie Mouse      19
Name: english_name, dtype: int64

In [37]:
df.local_name.value_counts().head(5)

 Pluto           14
 Daisy            4
 Mickey Mouse     4
 Goofy            4
 Minnie Mouse     3
Name: local_name, dtype: int64

### Clean data

- delete non alpha characters
- convert to lower case

In [11]:
clean_data(df)
df.head()

2021-06-17 09:28:35,509 - INFO - __main__.clean_data - Cleaning done after 0.0 s


,id,language,english_name,local_name,local_name_clean
0,1,Danish,"April, May, and June","Kylle, Pylle og Rylle",kyllepylleogrylle
1,2,Danish,Beagle Boys,Bjørnebanden,bjrnebanden
2,3,Danish,Big Bad Wolf,Store stygge ulv,storestyggeulv
3,4,Danish,Black Pete,Sorteper,sorteper
4,5,Danish,Chip 'n Dale,Chip og Chap,chipogchap


### LSH

In [69]:
# parameters
threshold = 0.5
num_perm = 120
weights = (0.5, 0.5)
nr_ngrams = 4

In [70]:
# run matching
matched_df = lsh_match_name(df, threshold, num_perm, weights, nr_ngrams)
matched_df['is_match'] = np.where(matched_df.english_name == matched_df.match_english_name, True, False)

2021-06-17 09:53:20,851 - INFO - __main__.lsh_match_name - Creating minhashes...
2021-06-17 09:53:21,580 - INFO - __main__.lsh_match_name - Querying minhashes...


In [71]:
# sample of matched characters where they are actually identical
matched_df[matched_df.is_match].head()

,id,local_name,english_name,match_id,match_local_name,match_english_name,is_match
373,374,Andeby,Duckburg,12,Andeby,Duckburg,True
387,388,Madam Mim,Mad Madam Mim,24,Madam Mim,Mad Madam Mim,True
183,184,Madam Mim,Mad Madam Mim,24,Madam Mim,Mad Madam Mim,True
546,547,Mickey Mouse,Mickey Mouse,25,Mickey Mouse,Mickey Mouse,True
61,62,Mickey Mouse,Mickey Mouse,25,Mickey Mouse,Mickey Mouse,True


In [72]:
# sample of matched characters where they are actually NOT identical - false positives
matched_df[~matched_df.is_match].head()

,id,local_name,english_name,match_id,match_local_name,match_english_name,is_match
346,347,Edi,Little Helper,141,Gus,Gus Goose,False
460,461,00-Zéro,Double-O Duck,141,Gus,Gus Goose,False
277,278,Plútó,Pluto,141,Gus,Gus Goose,False
155,156,Gevatter Fuchs,Br'er Fox,155,Gevatter Bär,Br'er Bear,False
154,155,Gevatter Bär,Br'er Bear,156,Gevatter Fuchs,Br'er Fox,False


### Comparison to exact matches

In [73]:
# matches on english name
df_english_merge = df.merge(df, on='english_name')
df_english_merge = df_english_merge[df_english_merge.id_x != df_english_merge.id_y]

# matches on local name
df_local_merge = df.merge(df, on='local_name')
df_local_merge = df_local_merge[df_local_merge.id_x != df_local_merge.id_y]

# report
print(f"Number of matches using english name: {df_english_merge.shape[0]}")
print(f"Number of matches using cleaned local name: {df_local_merge.shape[0]}")
print(f"Number of matches using LSH: {matched_df.shape[0]}, false positives: {matched_df[~matched_df.is_match].shape[0]}")

Number of matches using english name: 7336
Number of matches using cleaned local name: 270
Number of matches using LSH: 534, false positives: 32


In [74]:
print(f"Precision: {round(matched_df[matched_df.is_match].shape[0] / matched_df.shape[0], 2)}")
print(f"Recall: {round(matched_df[matched_df.is_match].shape[0] / df_english_merge.shape[0], 2)}")

Precision: 0.94
Recall: 0.07


In [86]:
weights = (0.5, 0.5)

for nr_ngrams in [2,4,6]:
    for threshold in [0.5, 0.7, 0.9]:
        for num_perm in [100, 120, 400]:
            
            starttime = timeit.default_timer()

            matched_df = lsh_match_name(df, threshold, num_perm, weights, nr_ngrams)
            matched_df['is_match'] = np.where(matched_df.english_name == matched_df.match_english_name, True, False)
            print("--------------------")
            print(f"Params: nr ngrams: {nr_ngrams}, threshold: {threshold}, num_perm: {num_perm}")
            print(f"""Precision: {round(matched_df[matched_df.is_match].shape[0] / matched_df.shape[0], 2)}, Recall: {round(matched_df[matched_df.is_match].shape[0] / df_english_merge.shape[0], 2)}, Time: {round(timeit.default_timer() - starttime, 2)} s""")

--------------------
Params: nr ngrams: 2, threshold: 0.5, num_perm: 100
Precision: 0.77, Recall: 0.09, Time: 2.46 s
--------------------
Params: nr ngrams: 2, threshold: 0.5, num_perm: 120
Precision: 0.76, Recall: 0.09, Time: 2.55 s
--------------------
Params: nr ngrams: 2, threshold: 0.5, num_perm: 400
Precision: 0.9, Recall: 0.09, Time: 3.85 s
--------------------
Params: nr ngrams: 2, threshold: 0.7, num_perm: 100
Precision: 0.95, Recall: 0.06, Time: 2.41 s
--------------------
Params: nr ngrams: 2, threshold: 0.7, num_perm: 120
Precision: 0.96, Recall: 0.06, Time: 2.57 s
--------------------
Params: nr ngrams: 2, threshold: 0.7, num_perm: 400
Precision: 0.96, Recall: 0.07, Time: 3.75 s
--------------------
Params: nr ngrams: 2, threshold: 0.9, num_perm: 100
Precision: 1.0, Recall: 0.04, Time: 2.36 s
--------------------
Params: nr ngrams: 2, threshold: 0.9, num_perm: 120
Precision: 1.0, Recall: 0.04, Time: 2.42 s
--------------------
Params: nr ngrams: 2, threshold: 0.9, num_perm